# Virtual Bean Machine (VBM)

The __Bean Machine__, also known as __Galton Board__, has been popularised by Francis Galton. Typically a wooden board with a reservoir of beans (or marbles) on the top and a number of bins at the bottom. In-between are several rows of pins. 

As beans fall down the board, they hit pins, and end up in one of the bins at the board\'s bottom. Once many beans have gone through the machine, a distribution emerges that resembles the normal distribution (in fact, it is the binomial distribution). 

At the core of the Bean Machine is __a simple random process__: a bean hitting a pin. When this happens, it is equally likely that the bean goes left or right (50% each). Adding another row of pins to the Galton Board adds another iteration of this random process, where the bean again goes either left or right. The outcome of this process, and of previous iterations, determines where the bean lands on the bottom of the board.  

In [151]:
# Functions for VBM animation. The animation comprises two graphical elements, a set of pins (top) and the histogram where the beans land (bottom)
#  FUNCTIONS  ============================================
# - for sampling the path of N beans on their way through P rows of pins: flip_GaltonBoard
# - for drawing the pins and the path of a specific bean: printPinBall_base, plotPath
# - for animation (also draws the histogram): updateFig
# - helper function for graphical layout: scaleGaltonBoard
# ========================================================

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

# Calculate path and bin where a bean lands (looped for all beans in sample)
def flip_GaltonBoard(nRows, nBeans):
    allMeans = []; allSeq = [] 
    for trial in range(nBeans):
        thisSeq = np.random.choice([0,1], nRows)         # random binary sequence 
        thisMean = np.mean(thisSeq)                      # mean of sequence
        allMeans.append(thisMean)
        allSeq.append(thisSeq)
    return allMeans, allSeq

# helper function for data layout (on every iteration). To put the rows of pins above the histogram and scale the two elements 
def scaleGaltonBoard(xCoords, yCoords, xBean, yBean, nRows, yScale, yOffset):
    bins = np.linspace(0, 1, num=nRows+2, endpoint=True) # binborders - the number of bins is nRows + 1
    # add yOffset - shift upwards
    xOffset = .5
    xScale = (1/(np.max(xCoords) - np.min(xCoords))) * (bins[-2] - bins[1])        # pin locations on xaxis should match bin boundaries
    xCoords = [x * xScale + xOffset for x in xCoords]                              # scale / translate positions  
    yCoords = [x * yScale + yOffset for x in yCoords]                              # scale / translate positions    
    xBean = [x * xScale + xOffset for x in xBean]                                  # repeat for path of bean in trial X
    yBean = [x * yScale + yOffset for x in yBean]    
    return xCoords, yCoords, xBean, yBean
 
# determine pin layout (for all pins in board) 
def printPinBall_base(nRows):
# draw raster of pins on the board
    stepsize = 1 / nRows     
    pinIdx = [list(range(x+1)) for x in range(nRows)]                              # nested list contains the pin indices per row, subordinate list the row indices. Wxample: [[0], [0,1], [0,1,2], ...]
    xCoords=[]; yCoords = []
    for x, y in zip(pinIdx, np.linspace(0,-1, num = nRows, endpoint=True)):        #convert pinIdx to xCoords
        tempo = np.linspace(-stepsize*(len(x)-1), stepsize*(len(x)-1),len(x))
        xCoords.extend(tempo)
        yCoords.extend([y]*len(tempo))    
    return xCoords, yCoords
    
def plotPath(thisSeq, nRows): 
# draw path - sequence of hit pins (repeat for each bean) 
    stepsize =  1 / nRows     
    pinIdx = [list(range(x+1)) for x in range(nRows)]   
    currInd = 0; thisSeqInd = [0]                                                  # init: start with pin0 in row0
    for x in thisSeq:
        if x == 1:                                                                 # if bean goes right, incr. index. Otherwise keep index of previous row
            currInd += 1
        thisSeqInd.append(currInd)    
    yCoords = np.linspace(0,-1, num = nRows, endpoint=True)
    xCoords=[]; 
    for x, y in zip(pinIdx, thisSeqInd):                                           #convert pinIdx to xCoords for plotting
        tempo = np.linspace(-stepsize*(len(x)-1), stepsize*(len(x)-1),len(x))
        xCoords.append(tempo[y])    
    return xCoords, yCoords, thisSeqInd


def updateFig(i):
# animation function - fetches the data and plots it 
    plt.cla()
    xCoords, yCoords = printPinBall_base(nRows)
    xBean, yBean, _ = plotPath(b[i], nRows)
    xCoords, yCoords, xBean, yBean = scaleGaltonBoard(xCoords, yCoords, xBean, yBean, nRows, yScale, yOffset)
    plt.plot(xCoords, yCoords, 'o', c='gray', alpha = .5, markersize = markersize)
    plt.plot(xBean, yBean, 'ro-', markersize = markersize)
    patches = plt.hist(a[:i], bins = bins, alpha=0.5, facecolor = 'gray', histtype='bar', ec='black')[2]
    patches[np.max(np.where(bins[:-1] <= a[i]))].set_facecolor('r')    
    for x in bins:
        plt.plot([x,x], [0, yLim - yScale - 1], 'k-', alpha = .05)        
    plt.gca().set_ylim(0, yLim);
    plt.gca().set_xlim(0, 1);
    plt.axis('off')    


__________
__Run the VBM__: Choose your preferred settings and start the machine by running the cell below. Change __nRows__ to increase or decrease the iterations of the random process. Change __nBeans__ to vary the number of beans that run through the machine.  In the animation, the pins that are hit by a bean as it passes through the machine are marked in red. The bin where the bean lands after having been "processed" by the machine is also shown in red. 

In [187]:
%matplotlib qt        
# =========== SETTINGS ==========
nRows = 13
nBeans = 200
# ===============================

# RUN BEANMACHINE    
bins = np.linspace(0, 1, num=nRows+2, endpoint=True) # these are actually the binborders - the number of bins is nRows + 1
a, b = flip_GaltonBoard(nRows, nBeans)    
yScale = 40
markersize = 100 / nRows
yLim = 1 + np.max(np.histogram(a, nRows+1)[0] + yScale) # results with full sample, just to get the yaxis limits
yOffset = yLim - 1 
    
fig = plt.figure(figsize=(8,10))
myAnimation = animation.FuncAnimation(fig, updateFig, frames=range(len(a)), interval=100, repeat = False)
plt.show()

__Distribution produced by the VBM__ From the histogram (at the bottom of the animation) it is obvious that a bean is likely to end up in a bin that is just below the bean\'s starting point on its way down the board, but there is also some variability around the distribution\'s center. It is also obvious that the resulting distribution depends on the number of pin rows. If there are only a few rows of pins, the bins will be wider and there will be fewer bins. This is because the range of theoretically possible scores is smaller when there are fewer iterations of the random process. 

Binning is the main reason why the resulting __binomial distribution__ is discrete (as the output states of the VBM are finite). For example, with two rows of pins there are three bins, and thus, there are only three possible outcomes of running a bean through the machine: The bean could end up in the bin on the left, in the center, or on the right - but nothing in-between. In contrast, the __normal distribution__ is a continuous distribution with an infinite number of theoretically possible variable scores. It is possible to approximate this continuous normal distribution with a VBM that has many rows of pins (and many beans). 

__Another way to think of the Bean Machine__ Statistically it makes no difference whether you run a Bean Machine with P rows of pins or if you flip a __fair coin__ P times. The distribution is the same. Also, the range of possible scores is the same. For example, if we write down 0 for heads and 1 for tails, the mean outcome of flipping a coin 10 times would be 0.5. Analogously, if you used a Bean Machine with 10 rows of pins and coded a bean going left as 0 and right as 1, respectively, the mean score would again be 0.5 


________

__Boundaries__ The Bean Machine is not exactly a pinball machine. It does normally not happen that beans bounce off the (virtual) walls on the left or right of the board, so these boundaries do not limit the distribution by "interfering" with the random process. Rather, they mark the range of theoretically possible outcomes for P iterations of the random process. As there are X pins in the X-th row of the VBM, the range of theoretically possible scores is equal to the number of pin rows P + 1. Thus, a VBM with 10 rows of pins has 11 bins (and can produce 11 outcomes for each bean that it processes).

How can we be sure that the board boundaries do not interfere with the random process on a given iteration? Imagine that you flipped a coin 9 times and ended up with 9 times heads. Another heads on the 10th coin flip would lead to the extreme case where the final score is zero (all heads). However, whether the outcome of this 10th coin flip is heads or tails is independent of the fact that the lowest possible score is zero.

In [173]:
# Functions for Pinball Animation

def pinHitProba(b):
# returns probability of pins being hit by beans (as a proportion of all used beans). 
# Returns a list of lists, where the superordinate list represents the row and 
# the subordinate list contains the probability of pins being hit in this row 
# (normalized by all observations)    
    allSeqs = [plotPath(x, np.shape(b)[1])[2][:-1] for x in b] # pin indices for all trials
    allSeqsT = [list(x) for x in zip(*allSeqs)] # transpose list 
    thisRow = 0; binHitProbability=[]
    for pinsInRow in allSeqsT:
        binHitProbability.append([pinsInRow.count(x) / len(pinsInRow) for x in range(thisRow+1)])
        thisRow += 1
    return binHitProbability

def updatePinball(i):
# simplified animation function - only pins, no histogram 
# brightness of pins reflect the propability of prior hits (by beans). 
# alpha is scared to focus on pins with high hit probs.
    plt.cla()
    hitProba = pinHitProba(b[:i+1])                                  # get probabilities of pins being hit up to current trial    
    cm = np.asarray([(1-x)**5 for sublist in hitProba for x in sublist]) # flatten list    
    plt.scatter(xCoords, yCoords, s = markersize,color='white', edgecolors='black', linewidths = 1)
    plt.scatter(xCoords, yCoords, alpha = cm , s = markersize, color='green')
    if i is not len(b)-1:
        xc,yc, _ = plotPath(b[i], nRows)
        plt.scatter(xc, yc, s = markersize, color='r')
        plt.plot(xc, yc, 'r-')
    plt.axis('off')        


__What are the chances that a bean ends up in a particular bin?__ With the empirical distribution that we had obtained with N beans and P rows of pins, we could easily calculate the probability that a bean would land in a particular bin. All we would have to do is to divide frequency by the overall number of observations (i.e., number of beans per bin divided by number of beans run through the machine).

__What are the chances that a particular pin is hit by a bean?__ Whether a particular pin is hit by a bean depends on the path the bean took before reaching the row where the pin is located. If we take the paths of all beans on previous trials into account, we can predict which pins are likely to be hit on the current trial. 

The animation below illustrates the hit probabilities per pin. These probabilities are updated from trial to trial, and they will change more drastically at an early stage when only few beans had been processes by the VBM. Every time a pin is hit, its brightness will be increased. If it hasn\'t been hit for a while, its brightness will decrease again. As the pin in the first row is hit on every trial, it will be white from the first trial on and never change it\'s brightness. (Note: For illustrative purposes, the brightness scaling is not linear but exponential).

To keep things simple, it makes sense to use a small number of pins and play the animation with a relatively slow speed. 

In [191]:
%matplotlib qt
# =========== SETTINGS ==========
nRows = 9
nBeans = 30
speed = 250 # time between frames in ms
markersize = 3000 / nRows
# ===============================

# RUN PINBALL
a, b = flip_GaltonBoard(nRows, nBeans)
xCoords, yCoords = printPinBall_base(nRows)

fig = plt.figure(figsize=(8,6))
plt.scatter(xCoords, yCoords, s = markersize,color='black', edgecolors='green', linewidths = .5)
plt.axis('off')    
myAnimation = animation.FuncAnimation(fig, updatePinball, frames=range(len(b)), interval=speed, repeat = False)
plt.show()